In [1]:
import sys
sys.path.insert(1, '../')

In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import os
import yaml
from pydoc import locate

In [69]:
def get_log_regression():
    return LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', C=1.0)

In [4]:
## define model id
model_id = '20200309-180350'
model_path = os.path.join('../logs', model_id, 'train', 'checkpoints')

In [6]:
def _load_stl10(prefix="train"):
    X_train = np.fromfile('../data/stl10_binary/' + prefix + '_X.bin', dtype=np.uint8)
    y_train = np.fromfile('../data/stl10_binary/' + prefix + '_y.bin', dtype=np.uint8)

    X_train = np.reshape(X_train, (-1, 3, 96, 96))
    X_train = np.transpose(X_train, (0, 3, 2, 1))
    print("{} images".format(prefix))
    print(X_train.shape)
    print(y_train.shape)
    y_train -= 1
    return X_train, y_train

In [7]:
# load STL-10 train data
X_train, y_train = _load_stl10("train")

# load STL-10 test data
X_test, y_test = _load_stl10("test")

train images
(5000, 96, 96, 3)
(5000,)
test images
(8000, 96, 96, 3)
(8000,)


In [8]:
batch_size=256

In [9]:
print("Training data")
print(X_train.shape)
print(y_train.shape)
print("Testing data")
print(X_test.shape)
print(y_test.shape)

Training data
(5000, 96, 96, 3)
(5000,)
Testing data
(8000, 96, 96, 3)
(8000,)


In [10]:
print(np.min(X_train), np.max(X_train))
print(np.min(X_test), np.max(X_test))

0 255
0 255


In [11]:
print(y_train[:5], y_test[:5])
# [5 1 7 2 1] [7 9 3 8 0]

[2 6 2 7 4] [7 8 6 1 4]


## Test protocol #1 PCA features

In [12]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [13]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train.reshape((X_train.shape[0], -1)))

pca = PCA(n_components=128)

X_train_pca = pca.fit_transform(scaler.transform(X_train.reshape(X_train.shape[0], -1)))
X_test_pca = pca.transform(scaler.transform(X_test.reshape(X_test.shape[0], -1)))

print("PCA features")
print(X_train_pca.shape)
print(X_test_pca.shape)

PCA features
(5000, 128)
(8000, 128)


In [14]:
clf = get_log_regression()
clf.fit(X_train_pca, y_train)
print("PCA feature evaluation")
print("Train score:", clf.score(X_train_pca, y_train))
print("Test score:", clf.score(X_test_pca, y_test))

# PCA feature evaluation
# Train score: 0.4592
# Test score: 0.3632

PCA feature evaluation
Train score: 0.4314
Test score: 0.3625


In [15]:
os.path.join(model_path, "baseline_encoder.py")

'../logs/20200309-180350/train/checkpoints/baseline_encoder.py'

In [16]:
from models.cnn_small import SmallCNN
from models.resnet_simclr import ResNetSimCLR
model = ResNetSimCLR(input_shape=(96,96,3), out_dim=256)

In [17]:
model(tf.ones((1,96,96,3)))
model.load_weights(os.path.join(model_path, 'model.h5'))

In [18]:
def next_batch(X, y, batch_size):
    for i in range(0, X.shape[0], batch_size):
        X_batch = X[i: i+batch_size] / 255.
        y_batch = y[i: i+batch_size]
        yield X_batch.astype(np.float32), y_batch

## Protocol #2 Linear separability evaluation

In [19]:
X_train_feature = []

for batch_x, batch_y in next_batch(X_train, y_train, batch_size):
    features, _ = model(batch_x)
    X_train_feature.extend(features.numpy())
    
X_train_feature = np.array(X_train_feature)

print("Train features")
print(X_train_feature.shape)

Train features
(5000, 2048)


In [20]:
X_test_feature = []

for batch_x, batch_y in next_batch(X_test, y_test, batch_size):
    features, _ = model(batch_x)
    X_test_feature.extend(features.numpy())
    
X_test_feature = np.array(X_test_feature)

print("Test features")
print(X_test_feature.shape)

Test features
(8000, 2048)


In [70]:
clf = get_log_regression()

scaler = preprocessing.StandardScaler()
scaler.fit(X_train_feature)

clf.fit(scaler.transform(X_train_feature), y_train)

/home/thalles/anaconda3/envs/tf2_cpu/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
print("SimCLR feature evaluation")
print("Train score:", clf.score(scaler.transform(X_train_feature), y_train))
print("Test score:", clf.score(scaler.transform(X_test_feature), y_test))

# SimCLR feature evaluation
# Train score: 0.5946
# Test score: 0.5202

SimCLR feature evaluation
Train score: 1.0
Test score: 0.377875


In [72]:
y_train[:15]

array([1, 5, 1, 6, 3, 9, 7, 4, 5, 8, 0, 6, 0, 8, 7], dtype=uint8)

In [73]:
clf.predict(scaler.transform(X_train_feature))[:15]

array([1, 5, 1, 6, 3, 9, 7, 4, 5, 8, 0, 6, 0, 8, 7], dtype=uint8)

In [74]:
y_test[:15]

array([6, 7, 5, 0, 3, 1, 1, 1, 4, 4, 0, 0, 4, 0, 1], dtype=uint8)

In [75]:
clf.predict(scaler.transform(X_test_feature))[:15]

array([6, 7, 1, 0, 5, 3, 2, 4, 9, 0, 1, 0, 4, 0, 1], dtype=uint8)